<a href="https://colab.research.google.com/github/yadavrishikesh/Deep-Learning-Slides-Code/blob/main/code/DL_OptimizationAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Binary Classification and Optimization Algorithms Comparison

In this notebook, we will:

1. Simulate a synthetic binary classification dataset.
2. Implement logistic regression from scratch.
3. Train the model using several optimization algorithms:
   - Gradient Descent (GD)
   - Stochastic Gradient Descent (SGD)
   - Newton-Raphson
   - Adam
4. Compare convergence and final classification performance.

## 1. Import Libraries
We need `numpy` for numerical computations and `matplotlib` for plotting convergence.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)  # ensure reproducibility


## 2. Generate Synthetic Binary Classification Data
We simulate two classes with 2D features. Each class is sampled from a Gaussian distribution.
We also add a bias term (intercept) to the feature matrix.
